In [ ]:
#https://docs.djangoproject.com/en/3.1/ref/models/querysets/

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from users.models import Customer

In [5]:
# 나이 30의 이름만
Customer.objects.filter(age=30).values('first_name')

<QuerySet [{'first_name': '영환'}, {'first_name': '보람'}, {'first_name': '은영'}, {'first_name': '상현'}, {'first_name': '정훈'}, {'first_name': '서연'}, {'first_name': '정식'}, {'first_name': '영수'}, {'first_name': '예진'}, {'first_name': '은지'}, {'first_name': '준영'}, {'first_name': '숙자'}, {'first_name': '현주'}, {'first_name': '성수'}, {'first_name': '상훈'}, {'first_name': '민재'}, {'first_name': '지아'}, {'first_name': '정순'}, {'first_name': '지원'}, {'first_name': '하은'}, '...(remaining elements truncated)...']>

In [6]:
print(Customer.objects.filter(age=30).values('first_name').query)

SELECT "users_customer"."first_name" FROM "users_customer" WHERE "users_customer"."age" = 30


In [9]:
# 나이가 30 이상인 사람의 인원 수  COL __gte, __gt, __lte, __lt
Customer.objects.filter(age__gte=30).count()

414

In [10]:
# 미성년자의 인원수
Customer.objects.filter(age__lt=20).count()

213

In [20]:
# 30 이면서 동시에 김씨인 사람 수
# Customer.objects.filter(age=30, last_name='김').values('last_name','first_name')
Customer.objects.filter(age=30, last_name='김').count()

6

In [21]:
# 30이상이면서 동시에 김씨인 사람 수
Customer.objects.filter(age__gte=30, last_name='김').count()

112

In [25]:
# 30이상이거나 동시에 김씨인 사람 수
from django.db.models import Q
Customer.objects.filter(Q(age__gte=30)|Q(last_name='강')).count()

425

In [24]:
(Customer.objects.filter(last_name='강') | Customer.objects.filter(age__gte=30)).count()

425

In [26]:
# '%' => 무엇이든, '_' => 반드시 그 자리에
# django orm 에서는 %만 사용 가능, 그 이상으로 패턴 비교를 하려면, 아예 정규표현식(regex)를 사용한다.
# 'python requlation 휴대폰 전화번호' 검색: 책 한권 분량 글 볼 수 있다
# https://regexr.com/
# 02- 로 시작하는 전화번호 개수 (02-%)
Customer.objects.filter(phone__startswith='02-').count()

249

In [29]:
print(Customer.objects.filter(phone__startswith='02-').query)

SELECT "users_customer"."id", "users_customer"."first_name", "users_customer"."last_name", "users_customer"."age", "users_customer"."country", "users_customer"."phone", "users_customer"."balance" FROM "users_customer" WHERE "users_customer"."phone" LIKE 02-% ESCAPE '\'


In [30]:
# 강원도민 중 황씨의 이름
Customer.objects.filter(country='강원도', last_name='황').values('first_name')

<QuerySet [{'first_name': '은정'}, {'first_name': '혜진'}, {'first_name': '경수'}, {'first_name': '준영'}]>

In [34]:
# 나이가 가장 많은 사람 10명 (-로 내림차순)
Customer.objects.order_by('-age')[:10]
# 이러면 다 꺼내와서 그 중 10개만 고른 거니까 좀 쓸데없는 짓 한거 아냐? 
# 아냐아냐 ORM은 일 좀 잘해서 개떡같이 물어봐도 잘 대답해주는 감이 없잖아 있지
# => 자동으로 ORM이 LIMIT, OFFSET 을 처리해준다.

<QuerySet [<Customer: Customer object (1)>, <Customer: Customer object (4)>, <Customer: Customer object (28)>, <Customer: Customer object (53)>, <Customer: Customer object (65)>, <Customer: Customer object (123)>, <Customer: Customer object (138)>, <Customer: Customer object (172)>, <Customer: Customer object (182)>, <Customer: Customer object (186)>]>

In [38]:
# 최빈 10명
Customer.objects.order_by('balance')[:10].values('balance')

<QuerySet [{'balance': 150}, {'balance': 150}, {'balance': 150}, {'balance': 150}, {'balance': 150}, {'balance': 160}, {'balance': 160}, {'balance': 160}, {'balance': 160}, {'balance': 170}]>

In [40]:
# 최빈 중 최연장자 10명
Customer.objects.order_by('balance', '-age')[:10].values('balance','age')

<QuerySet [{'balance': 150, 'age': 33}, {'balance': 150, 'age': 32}, {'balance': 150, 'age': 32}, {'balance': 150, 'age': 28}, {'balance': 150, 'age': 20}, {'balance': 160, 'age': 36}, {'balance': 160, 'age': 32}, {'balance': 160, 'age': 25}, {'balance': 160, 'age': 23}, {'balance': 170, 'age': 37}]>

In [41]:
# 최연장자 중 최빈 10명
Customer.objects.order_by('-age', 'balance')[:10].values('balance','age')

<QuerySet [{'balance': 190, 'age': 40}, {'balance': 370, 'age': 40}, {'balance': 550, 'age': 40}, {'balance': 620, 'age': 40}, {'balance': 630, 'age': 40}, {'balance': 720, 'age': 40}, {'balance': 740, 'age': 40}, {'balance': 760, 'age': 40}, {'balance': 870, 'age': 40}, {'balance': 960, 'age': 40}]>

In [43]:
# 성/ 이름 ㄱㄴㄷ 정렬 내림차순
Customer.objects.order_by('-last_name', '-first_name').values()[4]

{'id': 912,
 'first_name': '준영',
 'last_name': '황',
 'age': 16,
 'country': '강원도',
 'phone': '02-1030-5139',
 'balance': 6500}

# Annotate

In [51]:
from django.db.models import Count
from django.db.models.functions import Concat
Customer.objects.annotate(full_name=Concat('first_name', 'last_name')).values()[0]

{'id': 1,
 'first_name': '정호',
 'last_name': '유',
 'age': 40,
 'country': '전라북도',
 'phone': '016-7280-2855',
 'balance': 370,
 'full_name': '정호유'}

In [53]:
Customer.objects.annotate(Count('country')).values()[1]

{'id': 2,
 'first_name': '경희',
 'last_name': '이',
 'age': 36,
 'country': '경상남도',
 'phone': '011-9854-5133',
 'balance': 5900,
 'country__count': 1}

In [ ]:
articles = Article.objects.all()
for article in articles:
    print(f'{article.title} {article.comment_set.count()}'')

In [55]:
Article.objects.annotate(Count('comment')).values('title','comment__count')
for article in articles:
    print(f"{article.get('title')} {article.comment__count}")

NameError: name 'Article' is not defined